In [23]:
import numpy as np
import pandas as pd

# 1

In [24]:
np.random.seed(18182)
data = np.random.rand(10, 5)
df = pd.DataFrame(data)
for ind, row in df.iterrows():
    print(ind, row[row > 0.3].mean())

0 0.6083059700067814
1 0.6088307437641535
2 0.7888297918136683
3 0.7369399595785642
4 0.6833692604632612
5 0.4511191641587042
6 0.6627503660522434
7 0.8020524900727327
8 0.6304840550112694
9 0.6479659053234821


# 2

In [25]:
df = pd.read_csv('data/wells_info.csv')
df['CompletionDate'] = pd.to_datetime(df['CompletionDate'])
df['SpudDate'] = pd.to_datetime(df['SpudDate'])
((df.CompletionDate - df.SpudDate)/np.timedelta64(1, 'M')).astype(int).head()

0     6
1     4
2    11
3     3
4     3
dtype: int32

# 3

In [26]:
df = pd.read_csv('data/wells_info_na.csv')
df.select_dtypes(include=np.number).items()
# сначала заполнить столбцы числового типа медианой
for col_name, col_data in df.select_dtypes(include=np.number).items():
    df[col_name] = df[col_name].fillna(np.nanmedian(df[col_name]))
# затем остальные модой
for col_name, col_data in df.select_dtypes(include=object).items():
    df[col_name] = df[col_name].fillna(df[col_name].mode()[0])
print(df.select_dtypes(object))

  CompletionDate      FirstProductionDate       formation        BasinName  \
0     2015-09-13  2015-09-01 00:00:00.000          CODELL     DENVER BASIN   
1     2015-09-07  2015-09-01 00:00:00.000        NIOBRARA     DENVER BASIN   
2     2014-01-07  2015-04-01 00:00:00.000        NIOBRARA     DENVER BASIN   
3     2014-01-07  2013-12-01 00:00:00.000     BONE SPRING    PERMIAN BASIN   
4     2015-10-10  2015-08-01 00:00:00.000  WOLFCAMP UPPER    PERMIAN BASIN   
5     2014-11-05  2014-11-01 00:00:00.000          BAKKEN     DENVER BASIN   
6     2014-04-27  2014-04-01 00:00:00.000          BAKKEN  WILLISTON BASIN   

      StateName CountyName  
0      COLORADO       WELD  
1      COLORADO       WELD  
2      COLORADO       WELD  
3    NEW MEXICO       EDDY  
4    NEW MEXICO        LEA  
5  NORTH DAKOTA   MCKENZIE  
6  NORTH DAKOTA   MCKENZIE  


# 4

In [27]:
# Liquid only
df_wells_info = pd.read_csv('data/wells_info.csv').set_index('API')
df_production = pd.read_csv('data/production.csv')
liq = df_production.groupby('API').sum(numeric_only=True)
liq12 = df_production.groupby('API').apply(lambda x: x.head(12).sum())
res = df_wells_info
res.insert(2, 'LiquidAllTime', liq[["Liquid"]])
res.insert(3, 'Liquid_12M', liq12[["Liquid"]])
res

,PermitDate,SpudDate,LiquidAllTime,Liquid_12M,CompletionDate,FirstProductionDate,operatorNameIHS,formation,BasinName,StateName,CountyName,LatWGS84,LonWGS84,BottomHoleLatitude,BottomHoleLongitude,LATERAL_LENGTH_BLEND,PROP_PER_FOOT,WATER_PER_FOOT
API,,,,,,,,,,,,,,,,,,
5005072170100,2014-05-06,2014-05-06,201325,139501,2014-12-02,2014-11-01 00:00:00.000,BURLINGTON RESOURCES O&G CO LP,NIOBRARA,DENVER BASIN,COLORADO,ARAPAHOE,39.684606,-104.642128,39.68445,-104.60557,9005.0,994.6866,591.800400
5123377130000,2013-07-21,2013-10-06,34625,25118,2014-02-26,2014-02-01 00:00:00.000,SRC ENERGY INCORPORATED,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.509320,-104.780980,40.49692,-104.77859,4195.0,991.5857,628.632100
5123379280000,2013-09-08,2013-09-25,69009,49692,2014-09-07,2013-11-01 00:00:00.000,BONANZA CREEK ENERGY OPERATING CO LLC,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.335390,-104.363000,40.34780,-104.36863,4273.0,1000.2760,564.484100
5123379400000,2013-09-11,2014-12-05,81048,63292,2015-03-31,2015-03-01 00:00:00.000,NOBLE ENERGY INCORPORATED,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.152220,-104.530780,40.17445,-104.52932,7078.0,973.4437,824.002000
5123385820100,2014-01-07,2014-01-13,18050,15569,2014-04-23,2014-04-01 00:00:00.000,GREAT WESTERN OPERATING COMPANY LLC,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.508303,-104.868180,40.49558,-104.86757,3211.0,783.5919,603.141400
5123390320000,2014-02-27,2014-06-20,38365,27186,2014-10-30,2014-10-01 00:00:00.000,NOBLE ENERGY INCORPORATED,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.520989,-104.450861,40.52100,-104.46772,4259.0,1011.0440,698.681900
5123390440000,2014-02-28,2014-04-26,108032,80136,2014-08-16,2014-08-01 00:00:00.000,KERR-MCGEE OIL & GAS ONSHORE LP,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.118872,-104.789372,40.13079,-104.78716,4518.0,469.8262,798.167600
5123392690000,2014-04-14,2015-03-20,77472,51224,2015-08-03,2015-07-01 00:00:00.000,CRESTONE PEAK RESOURCES OPERATING LLC,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.147015,-104.898342,40.16105,-104.89340,4975.0,1267.8870,835.992800
5123399750000,2014-08-09,2014-11-14,39758,24843,2015-04-16,2015-04-01 00:00:00.000,KERR-MCGEE OIL & GAS ONSHORE LP,NIOBRARA,DENVER BASIN,COLORADO,WELD,40.263373,-104.727955,40.27426,-104.73307,4786.0,805.9642,1133.333000
